# AI-assisted SQL joins

<center><img src="fire_truck.png" width=600></center>

<br>

Welcome to today's webinar! We are going to explore how the London Fire Brigade's animal rescue efforts correlate with city-wide census data. 

### 📊 Our data

This project will use an aggregated and filtered (2011 only) version of the data from the [London Fire Brigade](https://data.london.gov.uk/dataset/animal-rescue-incidents-attended-by-lfb-2ogkn/) and the [2011 Census of the United Kingdom](https://data.london.gov.uk/dataset/census-2011-population-by-age-uk-districts-2jkq6/). In this case, the data is saved to a BigQuery database. You can find the tables in the `public` schema.

<br>

**Table 1: `animal_rescues_summary`**
| Column | Description |
|---|---|
| `area_code` | Geographic code. |
| `area` | Name of the London borough. |
| `nb_incidents` | Number of animal rescues. |
| `total_nb_units_deployed` | Number of fire units/vehicles used. |
| `total_service_duration_hrs` | Total time of all rescues. |
| `avg_hourly_cost` | The average cost per hour for the rescue. |

<br>

**Table 2: `census_data`**
| Column | Description |
|---|---|
| `area_code` | Geographic code. |
| `area` | Name of the UK borough. |
| `region` | Name of the region the borough belongs to. |
| `nb_residents` | Total population in the borough. |
| `age_0_to_14`, `age_15_to_24`, `age_25_to_64`, `age_over_65` | Age demographic counts. |


### 🤖 The AI Workflow
When solving a question, follow the following steps:
1. Think about the data you have and what you want to achieve.
2. Write a natural language prompt.
3. Understand the code generated by the AI.
4. Check the AI’s code for errors or hallucinations.
5. Run the code and verify the results.
6. Iterate

### Task 1: Recap & relational intro

Before we can combine data, we need to see how they relate. Relational databases work because tables share a common "key."

Let's look at the first 5 rows of our rescue data.

> 💡 **AI Prompting Tip**: Make sure to provide context about the data you are querying. Tell the AI what the files are so it remembers them for future steps.

**Solution prompt:**
> Look at the first 5 rows of animal_rescues_summary

Note that there are many prompts that will get the AI to write the correct SQL code. We'll provide some example solution prompts but you could have got the same output using a different prompt.

In [16]:
SELECT *
FROM animal_rescues_summary
LIMIT 5;

,area_code,area,nb_incidents,total_nb_units_deployed,total_service_duration_hrs,avg_hourly_cost
0,E09000028,Southwark,34,34,43,260.0
1,E09000022,Lambeth,28,27,31,260.0
2,E09000016,Havering,28,29,37,260.0
3,E09000031,Waltham Forest,28,33,49,260.0
4,E09000030,Tower Hamlets,28,29,36,260.0


Now, let's look at the census data. Do you see the column that exists in both?

**Solution prompt:**
> Look at the first 5 rows of census_data

In [17]:
SELECT *
FROM census_data
LIMIT 5

,area_code,area,region,nb_residents,age_0_to_14,age_15_to_24,age_25_to_64,age_over_65
0,E07000223,Adur,South East,61182,9820,6522,31406,13434
1,W06000001,Anglesey,Wales,69751,11088,7714,35303,15646
2,E07000026,Allerdale,North West,96422,15097,10400,50892,20033
3,E07000032,Amber Valley,East Midlands,122309,19993,13538,65999,22779
4,E07000224,Arun,South East,149518,21770,15374,73046,39328


### Task 2: Simple INNER JOIN & table aliases

In SQL, joining is the process of combining rows from two or more tables based on a related column between them. It allows you to query data that is spread across different parts of a database as if it were a single set.

The `INNER JOIN` is the most common type of join. It acts like a filter: it only returns rows where there is a match in both tables.
- If a row in Table A **has** a corresponding value in Table B, those rows are merged and included in the results.
- If a row in Table A **doesn't** have a match in Table B (or vice-versa), that row is excluded from the final output entirely.

<br>

![ing_inner_join](img_inner_join.png)

Let's combine the tables using the column that exists in both tables! 

> 💡 **AI Prompting Tip**: Unless you specify otherwise, AI will typically default to a `JOIN`, which is the same as `INNER JOIN`.

**Solution prompt:**
> Combine animal_rescues_summary with census_data using area

In [18]:
SELECT *
FROM animal_rescues_summary AS rescues
	INNER JOIN census_data AS population
		ON rescues.area_code = population.area_code

,area_code,area,nb_incidents,total_nb_units_deployed,total_service_duration_hrs,avg_hourly_cost,area_code,area,region,nb_residents,age_0_to_14,age_15_to_24,age_25_to_64,age_over_65
0,E09000002,Barking And Dagenham,15,15,19,260.0,E09000002,Barking And Dagenham,London,185911,45764,25638,95188,19321
1,E09000005,Brent,19,19,21,260.0,E09000005,Brent,London,311215,59036,42787,176716,32676
2,E09000004,Bexley,9,9,11,260.0,E09000004,Bexley,London,231997,44215,30660,119910,37212
3,E09000003,Barnet,26,27,35,260.0,E09000003,Barnet,London,356386,69884,44493,194577,47432
4,E09000006,Bromley,25,25,29,260.0,E09000006,Bromley,London,309392,56768,34031,166556,52037
5,E09000007,Camden,19,19,21,260.0,E09000007,Camden,London,220338,33660,34386,128315,23977
6,E09000008,Croydon,23,23,25,260.0,E09000008,Croydon,London,363378,74198,47157,197648,44375
7,E09000009,Ealing,25,26,28,260.0,E09000009,Ealing,London,338449,64854,43225,194143,36227
8,E09000010,Enfield,25,26,33,260.0,E09000010,Enfield,London,312466,65748,42237,165648,38833
9,E09000011,Greenwich,14,14,16,260.0,E09000011,Greenwich,London,254557,52254,36048,140139,26116


When we join tables, we often run into "Ambiguous Column" errors. Both tables have an `area` column. If we just ask for `area`, SQL won't know which one to pick.

Select the area from the incidents table, the number of incidents, and the number of residents. Leverage aliases to make it clear what table you are getting the data from.

**Solution prompt:**
> Combine area and number of incidents from animal_rescues_summary (rescues) with number of residents from census_data (census). 

In [19]:
SELECT  
	rescues.area AS rescue_area, 
	rescues.nb_incidents,
	population.nb_residents
FROM animal_rescues_summary AS rescues
	INNER JOIN census_data AS population
		ON rescues.area_code = population.area_code

,rescue_area,nb_incidents,nb_residents
0,Barking And Dagenham,15,185911
1,Brent,19,311215
2,Bexley,9,231997
3,Barnet,26,356386
4,Bromley,25,309392
5,Camden,19,220338
6,Croydon,23,363378
7,Ealing,25,338449
8,Enfield,25,312466
9,Greenwich,14,254557


Let's refine that list and order it by the areas with the most incidents.

**Solution prompt:**
> Order by most incidents

In [20]:
SELECT 
    rescues.area AS rescue_area, 
	rescues.nb_incidents,
	population.nb_residents
FROM animal_rescues_summary AS rescues
    INNER JOIN census_data AS population 
        ON rescues.area_code = population.area_code
ORDER BY nb_incidents DESC

,rescue_area,nb_incidents,nb_residents
0,Southwark,34,288283
1,Waltham Forest,28,258249
2,Tower Hamlets,28,254096
3,Havering,28,237232
4,Lambeth,28,303086
5,Islington,27,206125
6,Barnet,26,356386
7,Ealing,25,338449
8,Bromley,25,309392
9,Enfield,25,312466


### Task 3: The power of LEFT JOIN

An `INNER JOIN` only shows records that exist in **both** tables. But what if we want to show every area in the census data, and only add the rescue data if it’s available?

In that case we'd need a `LEFT JOIN`. This join is used when you want to keep all the records from the "left" table, even if there are no matches in the "right" table.
- If a match is found in the right table, the rows are combined normally.
- If there is no match, the columns from the right table will simply show as NULL.

<br>

![img_left_join](img_left_join.png)

Let's start by looking for rows in the census data that do not have a match in the rescue data. We'll use a `LEFT JOIN` for that.

> 💡 **AI Prompting Tip**: Use words like "all records from" or "everything in Table A" to signal a LEFT JOIN to the AI.

**Solution prompt:**
> Look at all records from census_data enriched with animal_rescues_summary

In [21]:
SELECT *
FROM census_data AS population
	LEFT JOIN animal_rescues_summary AS rescues
		ON population.area_code = rescues.area_code

,area_code,area,region,nb_residents,age_0_to_14,age_15_to_24,age_25_to_64,age_over_65,area_code,area,nb_incidents,total_nb_units_deployed,total_service_duration_hrs,avg_hourly_cost
0,None,None,South East,61182,9820,6522,31406,13434,None,None,NaN,NaN,NaN,NaN
1,None,None,Wales,69751,11088,7714,35303,15646,None,None,NaN,NaN,NaN,NaN
2,None,None,North West,96422,15097,10400,50892,20033,None,None,NaN,NaN,NaN,NaN
3,None,None,East Midlands,122309,19993,13538,65999,22779,None,None,NaN,NaN,NaN,NaN
4,None,None,South East,149518,21770,15374,73046,39328,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,None,None,None,83957,15977,11039,45069,11872,None,None,NaN,NaN,NaN,NaN
344,None,None,None,110535,18893,19663,55040,16939,None,None,NaN,NaN,NaN,NaN
345,None,None,None,137687,25652,15049,75733,21253,None,None,NaN,NaN,NaN,NaN
346,None,None,None,140664,28304,14314,76270,21776,None,None,NaN,NaN,NaN,NaN


Let's identify the areas without any rescue data. Select only the area, number of residents and number of incidents and filter for areas without rescue data.

**Solution prompt:**
> Select area, number of residents, and number of incidents. Filter for areas without animal_rescues_summary data.

In [29]:
SELECT 
	population.area, 
	population.nb_residents,
	rescues.nb_incidents
FROM census_data AS population
	LEFT JOIN animal_rescues_summary AS rescues 
		ON population.area_code = rescues.area_code
WHERE rescues.nb_incidents IS NULL

,area,nb_residents,nb_incidents
0,Adur,61182,NaN
1,Anglesey,69751,NaN
2,Allerdale,96422,NaN
3,Amber Valley,122309,NaN
4,Arun,149518,NaN
...,...,...,...
310,Stevenage,83957,NaN
311,Welwyn Hatfield,110535,NaN
312,East Hertfordshire,137687,NaN
313,St Albans,140664,NaN


### Task 4: Metrics from joined data

The individual boroughs would like to know how much money they are spending on rescues per resident. Let's combine population data with cost to find a ratio: **rescue cost per resident.**

> 💡 **AI Prompting Tip**: Describe the formula step-by-step. AI often struggles if you just say "calculate the cost." Instead, tell it which columns to multiply and what to name the result.

**Solution prompt:**
> Calculate the rescue cost per resident by multiplying the total service hours by the average hourly cost and then dividing by the total number of residents. Combine animal_rescues_summary with census_data using area_code.

In [23]:
SELECT 
    population.area, 
    (rescues.total_service_duration_hrs * rescues.avg_hourly_cost) / population.nb_residents AS cost_per_resident
FROM animal_rescues_summary AS rescues
	INNER JOIN census_data AS population 
		ON rescues.area_code = population.area_code

,area,cost_per_resident
0,Barking And Dagenham,0.026572
1,Brent,0.017544
2,Bexley,0.012328
3,Barnet,0.025534
4,Bromley,0.024370
5,Camden,0.024780
6,Croydon,0.017888
7,Ealing,0.021510
8,Enfield,0.027459
9,Greenwich,0.016342


Let's clean that up by only including two decimals and ordering it. Who pays the most per residents?

**Solution prompt:**
> Include only two decimals for cost_per_resident and order by cost_per_resident in descending order.

In [24]:
SELECT 
    population.area, 
    ROUND((rescues.total_service_duration_hrs * rescues.avg_hourly_cost) / population.nb_residents, 2) AS cost_per_resident
FROM animal_rescues_summary AS rescues
	INNER JOIN census_data AS population 
		ON rescues.area_code = population.area_code
ORDER BY cost_per_resident DESC

,area,cost_per_resident
0,City Of London,0.14
1,Waltham Forest,0.05
2,Islington,0.04
3,Havering,0.04
4,Tower Hamlets,0.04
5,Southwark,0.04
6,Barking And Dagenham,0.03
7,Barnet,0.03
8,Enfield,0.03
9,Hillingdon,0.03


Let's do something a little trickier. We'll do the same calculation, but now instead of by borough let's do it for the entire region. We'll need to use an aggregation to solve that question!

**Solution prompt:**
> Calculate the cost_per_resident by region

In [25]:
SELECT 
    population.region,
    ROUND((SUM(rescues.total_service_duration_hrs) * AVG(rescues.avg_hourly_cost)) / SUM(population.nb_residents), 2) AS cost_per_resident
FROM animal_rescues_summary AS rescues
	INNER JOIN census_data AS population 
		ON rescues.area_code = population.area_code
GROUP BY population.region
ORDER BY cost_per_resident DESC

,region,cost_per_resident
0,London,0.02


### Task 5: Filtering groups with HAVING

Remember: `WHERE` filters rows *before* they are grouped. `HAVING` filters the results *after* the aggregation.

<br>

![img_where_vs_having](img_where_vs_having.png)

Let's start by filtering for UK boroughs with more than 80,000 residents between 0 and 14 years old.

> 💡 **AI Prompting Tip**: Typically, the AI is pretty good at identifying the correct clause to use. If you do find it struggling, you can specify timing in your prompt. Tell the AI to filter "individual rows" to trigger a `WHERE` clause, and filter the "final calculated results" to trigger a `HAVING` clause.

**Solution prompt:**
> Filter in census_data for areas with more than 80000 residents between 0 and 14 years old

In [26]:
SELECT *
FROM census_data
WHERE age_0_to_14 > 80000

,area_code,area,region,nb_residents,age_0_to_14,age_15_to_24,age_25_to_64,age_over_65
0,E08000025,Birmingham,West Midlands,1073045,229920,174192,530720,138213
1,E08000032,Bradford,Yorkshire and The Humber,522452,114917,71047,267096,69392
2,E06000052,Cornwall,South West,532273,83411,60157,273464,115241
3,E06000047,County Durham,North East,513242,82102,68422,270373,92345
4,E08000034,Kirklees,Yorkshire and The Humber,422458,80535,56373,221400,64150
5,E08000035,Leeds,Yorkshire and The Humber,751485,128592,124709,388586,109598
6,E08000003,Manchester,North West,503127,91815,105300,258468,47544
7,E08000019,Sheffield,Yorkshire and The Humber,552698,94507,98443,274050,85698
8,E06000054,Wiltshire,South West,470981,84848,53917,246728,85488


Now, let's first aggregate and then filter. Find regions with more than 100,000 residents over 65 years old.

**Solution prompt:**
> Group by region and filter census_data for regions with more than 100000 residents over 65 years old

In [27]:
SELECT 
	region, 
	SUM(age_over_65) AS total_age_over_65
FROM census_data
GROUP BY region
HAVING SUM(age_over_65) > 100000

,region,total_age_over_65
0,South East,1482020
1,None,457456
2,East of England,865360
3,Wales,562544
4,North East,350936
5,North West,1171155
6,London,904749
7,South West,850353
8,Yorkshire and The Humber,874571
9,West Midlands,945975


Let's update that query to exclude the City Of London and order it appropriately.

**Solution prompt:**
> Also filter out any rows that have the "City Of London" area

In [28]:
SELECT 
	region, 
	SUM(age_over_65) AS total_age_over_65
FROM census_data
WHERE area <> 'City Of London'
GROUP BY region
HAVING SUM(age_over_65) > 100000
ORDER BY total_age_over_65 DESC

,region,total_age_over_65
0,South East,1482020
1,North West,1171155
2,West Midlands,945975
3,London,903714
4,Yorkshire and The Humber,874571
5,East of England,865360
6,South West,850353
7,East Midlands,757954
8,Wales,562544
9,None,457456


### 🚀 Moving forward

AI is an incredible "Junior Developer," but it needs you to be a clear "Lead Architect." As you continue your AI data journey, keep these strategies in mind:

- **Deconstruct to construct**: For complicated requests, break the query into pieces! Don't ask the AI for a 50-line query at once. Ask for the join first, verify the data, and then prompt the AI to add aggregations or complex math.
- **Be the editor**: Always scrutinize the AI’s output. Check that it’s using the correct table aliases and watch out for "hallucinated" column names that don't exist in your schema.
- **The error loop**: If the AI gives you an error, don't give up. Copy and paste the exact error message back into the prompt. The AI is often better at fixing its own mistakes than writing perfect code on the first try.
- **Context is king**: Always start your session by telling the AI exactly which tables and columns you have. The better the AI understands your "world," the better its code will be.

Great job today! You are now ready to blend disparate data sources and find hidden insights using the power of AI.